# FINAL Dataframe-  NLS -  Encyclopaedia Britannica

This notebook is going to merge all the data available that we have from our postprocess dataframes.

For each postprocess edition dataframe that we got from **Merging_EB_Terms.ipynb** (e.g. results_eb_1_edition_dataframe, results_eb_2_edition_dataframe, etc) we are going to add the information from the dataframe that we got from **Metadata_EB.ipynb** (metadata_eb_dataframe). 

The idea is to have per edition dataframe (and also supplement dataframe), all the information (which currently is splitted across several dataframes) in one. 


This notebook will store the final dataframes in results_NLS directory, and their name schema will be **final_eb_< NUM_EDITION >_dataframe**.

Per entry in these new dataframes we will have the following columns (see an example of one entry of the first edition):

- MMSID:                                              
- editionTitle:                          First edition, 1771, Volume 1, A-B
- editor:                                                  Smellie, William
- editor_date:                                                   1740-1795
- genre:                                                       encyclopedia
- language:                                                             eng
- termsOfAddress:                                                       NaN
- numberOfPages:                                                        832
- physicalDescription:               3 v., 160 plates : ill. ; 26 cm. (4to)
- place:                                                         Edinburgh
- publisher:              Printed for A. Bell and C. Macfarquhar; and so...
- referencedBy:           [Alston, R.C.  Engl. language III, 560, ESTC T...
- shelfLocator:                                                        EB.1
- editionSubTitle:        Illustrated with one hundred and sixty copperp...
- volumeTitle:            Encyclopaedia Britannica; or, A dictionary of ...
- year:                                                                1771
- volumeId:                                                       144133901
- metsXML:                                               144133901-mets.xml
- permanentURL:                            https://digital.nls.uk/144133901
- publisherPersons:                     [C. Macfarquhar, Colin Macfarquhar]
- volumeNum:                                                              1
- letters:                                                              A-B
- part:                                                                   0
- editionNum:                                                             1
- supplementTitle:                                                         
- supplementSubTitle:                                                      
- supplementsTo:                                                         []
- numberOfVolumes:                                                        6
- term:                                                                  OR
- definition:             A NEW A D I C T I A A, the name of several riv...
- relatedTerms:                                                          []
- header:                                           EncyclopaediaBritannica
- startsAt:                                                              15
- endsAt:                                                                15
- numberOfTerms:                                                         22
- numberOfWords:                                                         54
- positionPage:                                                           0
- typeTerm:                                                         Article
- altoXML:                                  144133901/alto/188082904.34.xml

### Loading the necessary libraries

In [188]:
import yaml
import numpy as np
import collections
import string
import copy

In [189]:
import pandas as pd
from yaml import safe_load
from pandas.io.json import json_normalize
from difflib import SequenceMatcher

### Functions

In [190]:
def add_metadata(df):
    
    mask = df["editionTitle"].str.contains('Volume')
    for i in range(0, len (mask)):
        if mask[i]:
            tmp=df.loc[i,'editionTitle'].split("Volume ")[1].split(",")
            if len(tmp)>=1:
                volume= tmp[0]
                letters = tmp[-1].replace(" ","")
                part_tmp = volume.split("Part ")
                if len(part_tmp)>1:
                    volume=part_tmp[0]
                    part = part_tmp[1]
    
                    try:
                        part = int(part)
                    except:
                        if "I" in part:
                            part = 1
                else:
                    part=0

                volume = int(volume)
                df.loc[i, "letters"] = letters
                df.loc[i,"part"] = part
                df.loc[i ,"volumeNum"] = volume
    return df

In [191]:
def create_dataframe(df_metadata, df_data, editionNum):
    
    
    appended_data = []
    df_m_ed = df_metadata[df_metadata['editionNum'] == editionNum]
    list_years=df_m_ed["year"].unique()
    
    for y in range(0, len(list_years)):
        df_d_year=df_data[df_data['year'] == list_years[y]].reset_index(drop=True)
        df_m_year = df_m_ed[df_m_ed['year'] == list_years[y]].reset_index(drop=True)
        list_vols = df_m_year["volumeNum"].unique()
        print("YEAR %s, list_vols %s" % (list_years[y], list_vols))
        for v in range(0,len(list_vols)):
            df_d_year_vl=df_d_year[df_d_year["volumeNum"] == list_vols[v]].reset_index(drop=True)
            df_m_year_vl = df_m_year[df_m_year['volumeNum']==list_vols[v]].reset_index(drop=True)
            list_parts = df_m_year_vl["part"].unique()
            print("VOL %s, list_parts are %s" % (list_vols[v], list_parts))
            for p in range(0, len(list_parts)):
                df_d_year_vl_p=df_d_year_vl[df_d_year_vl["part"] == list_parts[p]].reset_index(drop=True)
                df_m_year_vl_p=df_m_year_vl[df_m_year_vl["part"] == list_parts[p]].reset_index(drop=True)
            
                
                d_rows_year_vl_p=len(df_d_year_vl_p.index)-1
                print("number of data rows for year %s, vol %s and part %s is %s" % (list_years[y], list_vols[v], list_parts[p], d_rows_year_vl_p ))
        
                df_m_year_vl_p_f = df_m_year_vl_p.append([df_m_year_vl_p]*d_rows_year_vl_p,ignore_index=True).reset_index(drop=True)
                result = pd.concat([df_m_year_vl_p_f, df_d_year_vl_p], axis=1).reset_index(drop=True)
                print("Len of the new temporal concatenated df %s" %len(result.index))
                appended_data.append(result)
    
    if editionNum == 3:
        df_m_sup = df_m[df_m['supplementTitle'].str.contains("third")]
        list_years=df_m_sup["year"].unique()
        
        for y in range(0, len(list_years)):
            df_d_year=df_data[df_data['year'] == list_years[y]].reset_index(drop=True)
            df_m_year = df_m_sup[df_m_sup['year'] == list_years[y]].reset_index(drop=True)
            list_vols = df_m_year["volumeNum"].unique()
            print("SUP YEAR %s, list_vols %s" % (list_years[y], list_vols))
            for v in range(0,len(list_vols)):
                df_d_year_vl=df_d_year[df_d_year["volumeNum"] == list_vols[v]].reset_index(drop=True)
                df_m_year_vl = df_m_year[df_m_year['volumeNum']==list_vols[v]].reset_index(drop=True)
                list_parts = df_m_year_vl["part"].unique()
                print("SUP VOL %s, list_parts are %s" % (list_vols[v], list_parts))
                for p in range(0, len(list_parts)):
                    df_d_year_vl_p=df_d_year_vl[df_d_year_vl["part"] == list_parts[p]].reset_index(drop=True)
                    df_m_year_vl_p=df_m_year_vl[df_m_year_vl["part"] == list_parts[p]].reset_index(drop=True)
            
                
                    d_rows_year_vl_p=len(df_d_year_vl_p.index)-1
                    print("SUP number of data rows for year %s, vol %s and part %s is %s" % (list_years[y], list_vols[v], list_parts[p], d_rows_year_vl_p ))
        
                    df_m_year_vl_p_f = df_m_year_vl_p.append([df_m_year_vl_p]*d_rows_year_vl_p,ignore_index=True).reset_index(drop=True)
                    result = pd.concat([df_m_year_vl_p_f, df_d_year_vl_p], axis=1).reset_index(drop=True)
                    print("SUP Len of the new temporal concatenated df %s" %len(result.index))
                    appended_data.append(result)
                
                
    final_df = pd.concat(appended_data).reset_index(drop=True)
    final_df =  final_df.loc[:,~final_df.columns.duplicated()]
    return final_df

In [192]:
def create_dataframe_sup(df_metadata, df_data):

    appended_data=[]
    df_m_sup = df_m[df_m['supplementTitle'].str.contains("fourth")]
    list_years=df_m_sup["year"].unique()
        
    for y in range(0, len(list_years)):
        df_d_year=df_data[df_data['year'] == list_years[y]].reset_index(drop=True)
        df_m_year = df_m_sup[df_m_sup['year'] == list_years[y]].reset_index(drop=True)
        list_vols = df_m_year["volumeNum"].unique()
        print("SUP YEAR %s, list_vols %s" % (list_years[y], list_vols))
        for v in range(0,len(list_vols)):
            df_d_year_vl=df_d_year[df_d_year["volumeNum"] == list_vols[v]].reset_index(drop=True)
            df_m_year_vl = df_m_year[df_m_year['volumeNum']==list_vols[v]].reset_index(drop=True)
            list_parts = df_m_year_vl["part"].unique()
            print("SUP VOL %s, list_parts are %s" % (list_vols[v], list_parts))
            for p in range(0, len(list_parts)):
                df_d_year_vl_p=df_d_year_vl[df_d_year_vl["part"] == list_parts[p]].reset_index(drop=True)
                df_m_year_vl_p=df_m_year_vl[df_m_year_vl["part"] == list_parts[p]].reset_index(drop=True)
            
                
                d_rows_year_vl_p=len(df_d_year_vl_p.index)-1
                print("SUP number of data rows for year %s, vol %s and part %s is %s" % (list_years[y], list_vols[v], list_parts[p], d_rows_year_vl_p ))
        
                df_m_year_vl_p_f = df_m_year_vl_p.append([df_m_year_vl_p]*d_rows_year_vl_p,ignore_index=True).reset_index(drop=True)
                result = pd.concat([df_m_year_vl_p_f, df_d_year_vl_p], axis=1).reset_index(drop=True)
                print("SUP Len of the new temporal concatenated df %s" %len(result.index))
                appended_data.append(result)
                
                
    final_df = pd.concat(appended_data).reset_index(drop=True)
    final_df =  final_df.loc[:,~final_df.columns.duplicated()]
    return final_df

### 1. Loading the metadata dataframe

In [193]:
df_m= pd.read_json(r'../../results_NLS/metadata_eb_dataframe', orient="index")

### 2. Creating the NEW final dataframes for all Editions


Possible values:
- results_eb_4_5_6_suplement_dataframe
- results_eb_8_edition_dataframe
- results_eb_7_edition_dataframe
- results_eb_6_edition_dataframe
- results_eb_5_edition_dataframe
- results_eb_4_edition_dataframe
- results_eb_3_edition_dataframe
- results_eb_2_edition_dataframe
- results_eb_1_edition_dataframe


In [195]:
df_d = pd.read_json("../../results_NLS/results_eb_1_edition_dataframe", orient="index") 
len(df_d)

27080

In [196]:
df

,MMSID,editionTitle,editor,editor_date,genre,language,termsOfAddress,numberOfPages,physicalDescription,place,...,definition,relatedTerms,header,startsAt,endsAt,numberOfTerms,numberOfWords,positionPage,typeTerm,altoXML
0,9910796373804340,Volume 5,"Stewart, Dugald",1753-1828,encyclopedia,eng,NaN,898,"6 v. (648, 680, 724, 708, 584, 863 p.) ; 4to.",Edinburgh,...,"FIFTH, AND SIXTH EDITIONS HISTORY OF THE SCIEN...",[],A,9,9,3,8,0,Article,192547788/alto/192866785.34.xml
1,9910796373804340,Volume 5,"Stewart, Dugald",1753-1828,encyclopedia,eng,NaN,898,"6 v. (648, 680, 724, 708, 584, 863 p.) ; 4to.",Edinburgh,...,"FOR ARCHIBALD CONSTABLE AND COMPANY, EDINBURGH ;",[],A,9,9,3,7,1,Article,192547788/alto/192866785.34.xml
2,9910796373804340,Volume 5,"Stewart, Dugald",1753-1828,encyclopedia,eng,NaN,898,"6 v. (648, 680, 724, 708, 584, 863 p.) ; 4to.",Edinburgh,...,PART II. IN the farther prosecution of the pla...,"[OUS, LT:, T"";:, LORDSHIP, UNIVERSITY, OXFORD,...",DISSERTATIONFIRST,15,224,1,118202,0,Topic,192547788/alto/192866863.34.xml
3,9910796373804340,Volume 5,"Stewart, Dugald",1753-1828,encyclopedia,eng,NaN,898,"6 v. (648, 680, 724, 708, 584, 863 p.) ; 4to.",Edinburgh,...,"\ Note (A.) p. JO. ij TZ'T f , remark f d > as...","[WORKS, MONSIEUR, DIEU, MAIS, VESPRIT, SECT, S...",NOTESANDILLUSTRATIONS,225,273,1,30374,0,Topic,192547788/alto/192869593.34.xml
4,9910796373804340,Volume 5,"Stewart, Dugald",1753-1828,encyclopedia,eng,NaN,898,"6 v. (648, 680, 724, 708, 584, 863 p.) ; 4to.",Edinburgh,...,"Hungary, number of inhabitants in 1817 to be 8...","[SCHWARTNER, HUNGAR, PESTH, UNGARN, WIEN, HUNG...",HUNGARY,274,281,1,8295,0,Topic,192547788/alto/192870230.34.xml
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
309,9910796373804340,Volume 4,"Stewart, Dugald",1753-1828,encyclopedia,eng,NaN,844,"6 v. (648, 680, 724, 708, 584, 863 p.) ; 4to.",Edinburgh,...,"641 E; edition tone Swce of thtlanges. i;,male...",[SUPPLETNENT],HIMALAYAMOUNTAINS,769,772,1,4346,0,Topic,193696082/alto/193720546.34.xml
310,9910796373804340,Volume 4,"Stewart, Dugald",1753-1828,encyclopedia,eng,NaN,844,"6 v. (648, 680, 724, 708, 584, 863 p.) ; 4to.",Edinburgh,...,tended to Bristol and other great towns ; but ...,[],HOMHOM,775,776,1,1189,1,Article,193696082/alto/193720624.34.xml
311,9910796373804340,Volume 4,"Stewart, Dugald",1753-1828,encyclopedia,eng,NaN,844,"6 v. (648, 680, 724, 708, 584, 863 p.) ; 4to.",Edinburgh,...,"ome. was brought out, and was well received. T...",[],HOMHM,777,777,2,395,0,Article,193696082/alto/193720650.34.xml
312,9910796373804340,Volume 4,"Stewart, Dugald",1753-1828,encyclopedia,eng,NaN,844,"6 v. (648, 680, 724, 708, 584, 863 p.) ; 4to.",Edinburgh,...,"itchen average, furnish a dish and a half of s...",[],HORTICULTURE,779,796,1,19822,0,Topic,193696082/alto/193720676.34.xml


In [197]:
df_d.tail(3)["volumeNum"]

27077    3
27078    3
27079    3
Name: volumeNum, dtype: int64

Note: add_metadata function can take a while. 

In [198]:
df_d = add_metadata(df_d)

**IMPORTANT**

- Use create_dataframe for creating the final dataframe for all editions. This function needs to **indicate the NUMBER of the edition, in the last argument**. 
- Use create_dataframe_sup for creating the final dataframe for the supplements. 

In [200]:
df_final=create_dataframe(df_m, df_d, 1)
#df_final=create_dataframe_sup(df_m, df_d)

YEAR 1771, list_vols [1 2 3]
VOL 1, list_parts are [0]
number of data rows for year 1771, vol 1 and part 0 is 3854
Len of the new temporal concatenated df 3855
VOL 2, list_parts are [0]
number of data rows for year 1771, vol 2 and part 0 is 5035
Len of the new temporal concatenated df 5036
VOL 3, list_parts are [0]
number of data rows for year 1771, vol 3 and part 0 is 4550
Len of the new temporal concatenated df 4551
YEAR 1773, list_vols [1 2 3]
VOL 1, list_parts are [0]
number of data rows for year 1773, vol 1 and part 0 is 3947
Len of the new temporal concatenated df 3948
VOL 2, list_parts are [0]
number of data rows for year 1773, vol 2 and part 0 is 4994
Len of the new temporal concatenated df 4995
VOL 3, list_parts are [0]
number of data rows for year 1773, vol 3 and part 0 is 4694
Len of the new temporal concatenated df 4695


And these are the columns of our FINAL DATAFRAME

In [201]:
df_final.columns

Index(['MMSID', 'editionTitle', 'editor', 'editor_date', 'genre', 'language',
       'termsOfAddress', 'numberOfPages', 'physicalDescription', 'place',
       'publisher', 'referencedBy', 'shelfLocator', 'editionSubTitle',
       'volumeTitle', 'year', 'volumeId', 'metsXML', 'permanentURL',
       'publisherPersons', 'volumeNum', 'letters', 'part', 'editionNum',
       'supplementTitle', 'supplementSubTitle', 'supplementsTo',
       'numberOfVolumes', 'term', 'definition', 'relatedTerms', 'header',
       'startsAt', 'endsAt', 'numberOfTerms', 'numberOfWords', 'positionPage',
       'typeTerm', 'altoXML'],
      dtype='object')

### 3. Saving new final dataframes to disk

In [202]:
df_final.to_json(r'../../results_NLS/final_eb_1_dataframe', orient="index")

### 4. Loading new final dataframes to memory

In [203]:
df = pd.read_json('../../results_NLS/final_eb_1_dataframe', orient="index") 